In [ ]:
import backtrader as bt
import yfinance as yf
import pandas as pd
from strategy.ma_cross import MaCrossStrategy

In [31]:
# Cerebro engine
cerebro = bt.Cerebro()
cerebro.addstrategy(MaCrossStrategy)

# Download data
print("Downloading AAPL data from 2021-01-01 to 2023-01-01...")
df = yf.download('AAPL', start='2021-01-01', end='2023-01-01')
print("Data download completed")

# Prepare data
dataframe = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
dataframe.columns = ['open', 'high', 'low', 'close', 'volume']

# Create data feed
data = bt.feeds.PandasData(
    dataname=dataframe,
    datetime=None,
    open='open',
    high='high',
    low='low',
    close='close',
    volume='volume',
    openinterest=None
)

cerebro.adddata(data)

# Set initial capital and commission
initial_cash = 100000
cerebro.broker.setcash(initial_cash)
cerebro.broker.setcommission(commission=0.001)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timereturn')

# Run backtest
print("\n" + "="*60)
print("RUNNING BACKTEST - Moving Average Crossover Strategy")
print("="*60)
print(f"Initial Capital: ${initial_cash:,.2f}")
print(f"Commission Rate: 0.1% per trade")
print(f"Strategy: Buy when 20 SMA > 50 SMA, Sell when 20 SMA < 50 SMA")

results = cerebro.run()
strat = results[0]

# Get final portfolio value
final_value = cerebro.broker.getvalue()
net_profit = final_value - initial_cash
total_return = (final_value / initial_cash - 1) * 100

print("\n" + "="*60)
print("BACKTEST RESULTS SUMMARY")
print("="*60)
print(f"Initial Capital:    ${initial_cash:,.2f}")
print(f"Final Portfolio:    ${final_value:,.2f}")
print(f"Net Profit/Loss:    ${net_profit:,.2f}")
print(f"Total Return:       {total_return:+.2f}%")

# Performance metrics
sharpe_ratio = strat.analyzers.sharpe.get_analysis()
drawdown = strat.analyzers.drawdown.get_analysis()
returns = strat.analyzers.returns.get_analysis()
trade_analysis = strat.analyzers.trades.get_analysis()

print("\nPERFORMANCE METRICS:")
print(f"Sharpe Ratio:       {sharpe_ratio.get('sharperatio', 0):.3f}")
print(f"Max Drawdown:       {drawdown.get('max', {}).get('drawdown', 0):.2f}%")
print(f"Max Drawdown Period: {drawdown.get('max', {}).get('len', 0)} days")
print(f"Annual Return:      {returns.get('rnorm100', 0):.2f}%")

# Trade statistics
print("\nTRADE STATISTICS:")
print(f"Total Trades:       {strat.trade_count}")
if strat.trade_count > 0:
    win_rate = (strat.win_count / strat.trade_count) * 100
    avg_profit = strat.total_profit / strat.trade_count
    print(f"Winning Trades:     {strat.win_count}")
    print(f"Losing Trades:      {strat.loss_count}")
    print(f"Win Rate:           {win_rate:.1f}%")
    print(f"Average Profit:     ${avg_profit:,.2f}")
    print(f"Total Profit:       ${strat.total_profit:,.2f}")

# Detailed trade analysis
if hasattr(trade_analysis, 'total') and trade_analysis.total:
    print(f"\nDETAILED TRADE ANALYSIS:")
    print(f"Total Closed Trades: {trade_analysis.total.closed}")
    print(f"Total Open Trades:   {trade_analysis.total.open}")
    if trade_analysis.total.closed > 0:
        print(f"Won Trades:          {trade_analysis.won.total}")
        print(f"Lost Trades:         {trade_analysis.lost.total}")
        print(f"Longest Win Streak:  {trade_analysis.streak.won.longest}")
        print(f"Longest Loss Streak: {trade_analysis.streak.lost.longest}")

# Individual trade results
if strat.trade_results:
    print(f"\nINDIVIDUAL TRADE RESULTS:")
    print("-" * 80)
    print(f"{'Entry Date':12} {'Exit Date':12} {'Entry':>8} {'Exit':>8} {'P/L %':>8} {'P/L $':>10} {'Days':>5}")
    print("-" * 80)
    
    for i, trade in enumerate(strat.trade_results, 1):
        profit_sign = '+' if trade['profit_abs'] > 0 else ''
        print(f"{trade['entry_date']} {trade['exit_date']} "
              f"${trade['entry_price']:7.2f} ${trade['exit_price']:7.2f} "
              f"{profit_sign}{trade['profit_pct']:6.1f}% "
              f"{profit_sign}${trade['profit_abs']:8.2f} "
              f"{trade['hold_days']:4}")

# Risk metrics
print(f"\nRISK METRICS:")
print(f"Final Return:       {total_return:.2f}%")
print(f"Profit Factor:      {trade_analysis.get('pnl', {}).get('gross', {}).get('profit', 0) / max(1, abs(trade_analysis.get('pnl', {}).get('gross', {}).get('loss', 1))):.2f}")

# Benchmark comparison (buy and hold)
buy_hold_return = (dataframe['close'].iloc[-1] / dataframe['close'].iloc[0] - 1) * 100
print(f"\nBENCHMARK COMPARISON:")
print(f"Strategy Return:    {total_return:.2f}%")
print(f"Buy & Hold Return:  {buy_hold_return:.2f}%")
print(f"Alpha:              {total_return - buy_hold_return:.2f}%")

print("\n" + "="*60)
print("BACKTEST COMPLETED SUCCESSFULLY")
print("="*60)

C:\Users\Gary\AppData\Local\Temp\ipykernel_17004\1402562742.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('AAPL', start='2021-01-01', end='2023-01-01')
[*********************100%***********************]  1 of 1 completed

Data download completed

RUNNING BACKTEST - Moving Average Crossover Strategy
Initial Capital: $100,000.00
Commission Rate: 0.1% per trade
Strategy: Buy when 20 SMA > 50 SMA, Sell when 20 SMA < 50 SMA

BACKTEST RESULTS SUMMARY
Initial Capital:    $100,000.00
Final Portfolio:    $100,001.37
Net Profit/Loss:    $1.37
Total Return:       +0.00%

PERFORMANCE METRICS:
Sharpe Ratio:       -54.089
Max Drawdown:       0.04%
Max Drawdown Period: 459 days
Annual Return:      0.00%

TRADE STATISTICS:


AttributeError: 'Lines_LineSeries_LineIterator_DataAccessor_Strateg' object has no attribute 'trade_count'